In [11]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs


In [2]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df = spark.read.parquet('../output/ifood/dataset_1/info_session/ground_truth/')\
        .sort('click_timestamp', 'account_id')
df.show(20)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|6df9ebc5-64c8-47b...|e3ff3dc9-d5c0-4f7...|92c5c3a6-40be-4a8...|2019-06-01 00:03:44|  1|  2019-06-01|
|381c7804-12a2-4cf...|2c11bfb6-6fe1-426...|92c5c3a6-40be-4a8...|2019-06-01 00:12:44|  1|  2019-06-01|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:36:01|  1|  2019-06-01|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:38:10|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:35|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:35|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:3

In [3]:
df = df.filter(df.buy == 1)

In [4]:
df.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- click_timestamp: timestamp (nullable = true)
 |-- buy: integer (nullable = true)
 |-- dt_partition: string (nullable = true)



In [5]:
df.count()

10000

In [35]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_all = spark.read.parquet('../output/ifood/dataset_3/info_session/merged/MergeIteractionDatasetTask_2000_output_evaluatio_0_0_a716710c4d_test_gt/')\
        .sort('click_timestamp', 'account_id')
df_all.filter(df_all.buy==1).count()

2000

In [26]:
df = df_all
# Drop nan and duplicates
df = df.na.drop().dropDuplicates()

# filters only the purchase iteration when you have both (purchase and visit) 
# at the same time
df_buy        = df.filter(df.buy==1)

df_only_visit = df.groupBy(['session_id','account_id','merchant_id', 
                        'click_timestamp', 'dt_partition'])\
                .sum().withColumnRenamed("sum(buy)", "buy").filter(col("buy") == 0)

    # union buys with visits without visitsBuy
df = df_buy.select(df.columns)\
        .union(df_only_visit.select(df.columns))
df.count()

3435

In [37]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_test = spark.read.parquet('../output/ifood/dataset_3/session_test_1995.00_s=2000/')
df_test.count()

1995

In [42]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_test = spark.read.parquet('../output/ifood/dataset_3/session_test_2000.00_s=6000/')
df_test.count()

2000

In [32]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_test = spark.read.parquet('../output/ifood/dataset_3/indexed_orders_test_data_2000.00_k=0_s=6000_f=1.parquet')
df_test.count()

599

In [19]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_test = spark.read.parquet('../output/ifood/dataset_1/indexed_orders_test_data_0.25_k=0_s=8000_f=1.parquet')
df_test.count()

623

In [20]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df_test = spark.read.parquet('../output/ifood/dataset_1/indexed_orders_test_data_0.20_k=0_s=10000_f=1.parquet')
df_test.count()

516

In [8]:
df_test.filter(df_test.buy == 0).count()

0

In [13]:
df_all.filter(df_all.buy == 1).count()

2751

In [6]:
df = df.distinct()
print(df.count())

df = df.na.drop()
print(df.count())

7594
6929


In [4]:
batch = 2000

In [37]:
df_merge_1 = spark.read.parquet('../output/ifood/dataset_5/info_session/merged/MergeIteractionDatasetTask_2000_output_evaluatio_0_0_b2914d28db/')\
                .sort('click_timestamp')#.limit(batch)
df_merge_1.limit(10).toPandas()

,session_id,account_id,merchant_id,click_timestamp,buy,dt_partition
0,6df9ebc5-64c8-47bc-8599-1d58c7fbe433,e3ff3dc9-d5c0-4f72-915b-42d9774a876c,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:03:44,1,2019-06-01
1,381c7804-12a2-4cff-8aa1-e49c7bb0a442,2c11bfb6-6fe1-426e-af3c-f51f50a36320,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:12:44,1,2019-06-01
2,3e50b1d0-9e31-4495-88fa-c685b64f31eb,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:36:01,1,2019-06-01
3,3e50b1d0-9e31-4495-88fa-c685b64f31eb,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:38:10,1,2019-06-01
4,1f0206f0-97f4-457b-a589-0dd99351ade9,d844438e-8c06-46ef-9f21-0d88efafe1a1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:40:35,1,2019-06-01
5,1f0206f0-97f4-457b-a589-0dd99351ade9,d844438e-8c06-46ef-9f21-0d88efafe1a1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:40:35,3,2019-06-01
6,1f0206f0-97f4-457b-a589-0dd99351ade9,d844438e-8c06-46ef-9f21-0d88efafe1a1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:40:35,3,2019-06-01
7,1f0206f0-97f4-457b-a589-0dd99351ade9,d844438e-8c06-46ef-9f21-0d88efafe1a1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:41:47,3,2019-06-01
8,1f0206f0-97f4-457b-a589-0dd99351ade9,d844438e-8c06-46ef-9f21-0d88efafe1a1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:48:54,3,2019-06-01
9,be7318ae-2c5b-4433-8787-7b2f00d4f7cd,0778450e-76ca-4c19-a459-3b08cda8e4ba,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:49:58,3,2019-06-01


In [38]:
df_merge_1.count()

6113

In [41]:
d = df_merge_1.sort('click_timestamp', ascending=False).limit(batch)
d.filter(d.buy == 1).count()

2000

In [30]:
df_merge_1.filter(df_merge_1.buy == 1).count()#.count()/df_merge_1.count()

2005

In [31]:
df_merge_1.filter(df_merge_1.rhat_merchant_id.isNotNull()).count()

AttributeError: 'DataFrame' object has no attribute 'rhat_merchant_id'

In [85]:
df_merge_1.filter(df_merge_1.session_id == "0c07cf50-ca9e-4e90-8310-5bd2b9e9b2ec").show()

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|0c07cf50-ca9e-4e9...|05c4655e-9957-4e9...|a17b106f-1fe3-471...|2019-06-24 14:40:51|  0|  2019-06-24|
|0c07cf50-ca9e-4e9...|05c4655e-9957-4e9...|a17b106f-1fe3-471...|2019-06-24 14:43:46|  0|  2019-06-24|
+--------------------+--------------------+--------------------+-------------------+---+------------+



In [47]:
df_merge_1.filter(df_merge_1.rhat_merchant_id.isNull()).toPandas()

,session_id,account_id,merchant_id,click_timestamp,buy,dt_partition,old_buy,old_merchant_id,rhat_merchant_id
0,381c7804-12a2-4cff-8aa1-e49c7bb0a442,2c11bfb6-6fe1-426e-af3c-f51f50a36320,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:12:44,1,2019-06-01,1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,None
1,3e50b1d0-9e31-4495-88fa-c685b64f31eb,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:36:01,1,2019-06-01,1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,None
2,3e50b1d0-9e31-4495-88fa-c685b64f31eb,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 00:38:10,1,2019-06-01,1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,None
3,3e50b1d0-9e31-4495-88fa-c685b64f31eb,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-01 02:04:43,1,2019-06-01,1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,None
4,893449ae-f05b-4caf-b42a-4e895aa8e926,None,92c5c3a6-40be-4a81-8969-3505b7bf16cc,2019-06-02 15:05:22,1,2019-06-02,1,92c5c3a6-40be-4a81-8969-3505b7bf16cc,None
...,...,...,...,...,...,...,...,...,...
7833,5b45641e-e586-42b6-8d01-01983156d12f,None,a17b106f-1fe3-4714-8a86-aa8e18968503,2019-07-31 20:45:35,1,2019-07-31,1,a17b106f-1fe3-4714-8a86-aa8e18968503,None
7834,5b45641e-e586-42b6-8d01-01983156d12f,None,a17b106f-1fe3-4714-8a86-aa8e18968503,2019-07-31 20:45:41,1,2019-07-31,1,a17b106f-1fe3-4714-8a86-aa8e18968503,None
7835,5b45641e-e586-42b6-8d01-01983156d12f,None,a17b106f-1fe3-4714-8a86-aa8e18968503,2019-07-31 20:46:11,1,2019-07-31,1,a17b106f-1fe3-4714-8a86-aa8e18968503,None
7836,5b45641e-e586-42b6-8d01-01983156d12f,None,a17b106f-1fe3-4714-8a86-aa8e18968503,2019-07-31 20:46:26,1,2019-07-31,1,a17b106f-1fe3-4714-8a86-aa8e18968503,None


In [38]:
df_merge_1.filter(df_merge_1.rhat_merchant_id.isNotNull()).count()

1985

In [35]:
df_merge_1.count()

2000

In [14]:
df_merge_1.filter(df_merge_1.old_merchant_id.isNull()).count()

AttributeError: 'DataFrame' object has no attribute 'old_merchant_id'

In [15]:
df_merge_1.filter(df_merge_1.buy == 1).count()/df_merge_1.count()

1.0

In [25]:
df_merge_2 = spark.read.parquet('../output/ifood/dataset_4/info_session/merged/MergeIteractionDatasetTask_output_evaluatio_0_2000_6c557f7a56/')\
                .sort('click_timestamp')#\
                #.limit(2*batch)
df_merge_2.show(10)

+--------------------+--------------------+--------------------+-------------------+---+------------+-------+--------------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|old_buy|     old_merchant_id|
+--------------------+--------------------+--------------------+-------------------+---+------------+-------+--------------------+
|6df9ebc5-64c8-47b...|e3ff3dc9-d5c0-4f7...|92c5c3a6-40be-4a8...|2019-06-01 00:03:44|  1|  2019-06-01|      1|92c5c3a6-40be-4a8...|
|381c7804-12a2-4cf...|2c11bfb6-6fe1-426...|92c5c3a6-40be-4a8...|2019-06-01 00:12:44|  1|  2019-06-01|      1|92c5c3a6-40be-4a8...|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:36:01|  1|  2019-06-01|      1|92c5c3a6-40be-4a8...|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:38:10|  1|  2019-06-01|      1|92c5c3a6-40be-4a8...|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:35

In [26]:
df_merge_2.count()

10176

In [12]:
df_merge_2.filter(df_merge_2.buy == 1).count()/df_merge_2.count()

0.5035

In [25]:
df_merge_2.filter(df_merge_2.buy == 0).count()

706

In [29]:
df_merge_3 = spark.read.parquet('../output/ifood/dataset_1/info_session/merged/MergeIteractionDatasetTask_output_evaluatio_5_3000_661aed9819/')\
                .sort('click_timestamp')\
                .limit(3*batch)
df_merge_3.show(10)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|6df9ebc5-64c8-47b...|e3ff3dc9-d5c0-4f7...|92c5c3a6-40be-4a8...|2019-06-01 00:03:44|  1|  2019-06-01|
|381c7804-12a2-4cf...|2c11bfb6-6fe1-426...|92c5c3a6-40be-4a8...|2019-06-01 00:12:44|  1|  2019-06-01|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:36:01|  1|  2019-06-01|
|3e50b1d0-9e31-449...|                null|92c5c3a6-40be-4a8...|2019-06-01 00:38:10|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:35|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:35|  1|  2019-06-01|
|1f0206f0-97f4-457...|d844438e-8c06-46e...|92c5c3a6-40be-4a8...|2019-06-01 00:40:3

In [31]:
df_merge_3.filter(df_merge_3.buy == 1).count()/df_merge_3.count()

0.5576666666666666

In [115]:
df_info_session.unionAll(df_info_session2).except(df_info_session.intersect(df_info_session2)).count()



SyntaxError: invalid syntax (<ipython-input-115-726b111c9009>, line 1)

In [5]:
df_info_session.count()

7255085

In [36]:
df_mearchant = spark.read.csv('../output/ifood/dataset/merchants_for_session_train_0.10_k=5_s=-1.csv',header=True, inferSchema=True)
df_mearchant = df_mearchant.select('merchant_idx', 'merchant_id')
df_mearchant.show(2)

+------------+--------------------+
|merchant_idx|         merchant_id|
+------------+--------------------+
|           0|0050c482-ca8e-4d3...|
|           1|025aa016-d05f-443...|
+------------+--------------------+
only showing top 2 rows



In [6]:
df_order = spark.read.parquet('../output/ifood/dataset/indexed_orders_test_data_0.10_k=5_s=-1.parquet').sort('click_timestamp')
df_order.select('session_id', 'account_id', 'click_timestamp', 'merchant_idx').show(2)

+--------------------+--------------------+-------------------+------------+
|          session_id|          account_id|    click_timestamp|merchant_idx|
+--------------------+--------------------+-------------------+------------+
|3295faa0-86f2-43c...|1eec4b73-01c5-4e3...|2019-07-24 11:27:58|          11|
|d77c2bdc-693f-411...|2405efc4-bcca-44c...|2019-07-24 11:27:59|         230|
+--------------------+--------------------+-------------------+------------+
only showing top 2 rows



In [12]:
# Create new column in your dataset
df_order = df_order.withColumn("new_timestamp", F.to_utc_timestamp(df_order.click_timestamp, str(local_tz)))
df_order.select('session_id', 'account_id', 'click_timestamp', 'new_timestamp', 'merchant_idx').sort('click_timestamp').show(2)

+--------------------+--------------------+-------------------+-------------------+------------+
|          session_id|          account_id|    click_timestamp|      new_timestamp|merchant_idx|
+--------------------+--------------------+-------------------+-------------------+------------+
|3295faa0-86f2-43c...|1eec4b73-01c5-4e3...|2019-07-24 11:27:58|2019-07-24 14:27:58|          11|
|d77c2bdc-693f-411...|2405efc4-bcca-44c...|2019-07-24 11:27:59|2019-07-24 14:27:59|         230|
+--------------------+--------------------+-------------------+-------------------+------------+
only showing top 2 rows



In [14]:
df_order_pd = pd.read_parquet('../output/ifood/dataset/indexed_orders_test_data_0.10_k=5_s=-1.parquet').sort_values('click_timestamp')
df_order_pd[['session_id', 'account_id', 'click_timestamp', 'merchant_idx']].head(2)

,session_id,account_id,click_timestamp,merchant_idx
6217,3295faa0-86f2-43c8-a64b-f0a658f7402f,1eec4b73-01c5-4e3d-8259-caa741c1ebe1,2019-07-24 14:27:58,11
25316,96213ce0-0f2a-46f5-802a-9989a0fe4cd2,54c1dc6f-0caf-4aa7-8204-baf3cc352cef,2019-07-24 14:27:59,1170


In [92]:
df_eval = spark.read.csv('../output/evaluation/EvaluateAutoEncoderIfoodModel/results/VariationalAutoEncoderTraining_selu____500_fc62ac744a_6534ac1232/orders_with_metrics.csv',header=True, inferSchema=True).sort('click_timestamp')
df_eval = df_eval.withColumn("click_timestamp", F.from_utc_timestamp(df_eval.click_timestamp, local_tz))
df_eval = df_eval.join(df_mearchant, df_eval.rhat_merchant_idx == df_mearchant.merchant_idx)
df_eval = df_eval.withColumnRenamed("merchant_id", "rhat_merchant_id")
df_eval = df_eval.withColumn("buy", lit(1))
df_eval.select('session_id', 'account_id', 'click_timestamp', 'rhat_merchant_id', 'rhat_merchant_idx', 'buy').show(2)

+--------------------+--------------------+-------------------+--------------------+-----------------+---+
|          session_id|          account_id|    click_timestamp|    rhat_merchant_id|rhat_merchant_idx|buy|
+--------------------+--------------------+-------------------+--------------------+-----------------+---+
|3295faa0-86f2-43c...|1eec4b73-01c5-4e3...|2019-07-24 11:27:58|15bc9da2-0f8c-4b2...|              826|  1|
|96213ce0-0f2a-46f...|54c1dc6f-0caf-4aa...|2019-07-24 11:27:59|2f905ea5-364f-49a...|             1170|  1|
+--------------------+--------------------+-------------------+--------------------+-----------------+---+
only showing top 2 rows



In [93]:
df_eval.count()

34940

In [95]:
joined = df_info_session.join(
    df_eval.select('session_id', 'account_id', 'click_timestamp', 'rhat_merchant_id', 'buy'), 
    ['session_id', 'account_id', 'click_timestamp', 'buy'], how='left').cache()

joined.show(2)

+--------------------+--------------------+-------------------+---+--------------------+------------+----------------+
|          session_id|          account_id|    click_timestamp|buy|         merchant_id|dt_partition|rhat_merchant_id|
+--------------------+--------------------+-------------------+---+--------------------+------------+----------------+
|0006707d-028e-450...|4d44c19a-0173-4d2...|2019-06-23 08:50:54|  0|47e4c365-5047-49a...|  2019-06-23|            null|
|0006ab6a-22f6-48a...|8be8733e-6171-4be...|2019-06-18 11:50:56|  0|3cc76a2d-794a-451...|  2019-06-18|            null|
+--------------------+--------------------+-------------------+---+--------------------+------------+----------------+
only showing top 2 rows



In [96]:
joined.filter(joined.rhat_merchant_id.isNotNull()).count()

57615

In [70]:
joined.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- click_timestamp: timestamp (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- buy: integer (nullable = true)
 |-- dt_partition: string (nullable = true)
 |-- rhat_merchant_id: string (nullable = true)



In [97]:
from pyspark.sql.functions import when, lit
from pyspark.sql.types import IntegerType

df = joined
#df = df.withColumn('acc', joined.merchant_id == joined.rhat_merchant_id).cache()
df = df.withColumn('buy_2', when(df.rhat_merchant_id.isNull(), df.buy).otherwise((joined.merchant_id == joined.rhat_merchant_id).cast(IntegerType())))
df = df.withColumn('merchant_id_2', when(df.rhat_merchant_id.isNull(), df.merchant_id).otherwise(df.rhat_merchant_id))

df = df.limit(5000).toPandas()
df.head()#.show(10)

,session_id,account_id,click_timestamp,buy,merchant_id,dt_partition,rhat_merchant_id,buy_2,merchant_id_2
0,0006707d-028e-450f-8b5f-82ca6afb6004,4d44c19a-0173-4d29-b0b1-a3ab3f18dd61,2019-06-23 08:50:54,0,47e4c365-5047-49a4-8f7c-37ffa2f15579,2019-06-23,None,0,47e4c365-5047-49a4-8f7c-37ffa2f15579
1,0006ab6a-22f6-48ab-9fd2-4b4f12eefd61,8be8733e-6171-4be5-9645-c29bb945fd13,2019-06-18 11:50:56,0,3cc76a2d-794a-4517-8ade-23ec49e6a381,2019-06-18,None,0,3cc76a2d-794a-4517-8ade-23ec49e6a381
2,0009f616-ff45-41be-9d3c-99ee55744b1a,30266171-963e-44a1-8861-3e281eb54e8e,2019-07-17 12:33:14,0,25036a46-d535-427f-b09a-c254e58429eb,2019-07-17,None,0,25036a46-d535-427f-b09a-c254e58429eb
3,0009f616-ff45-41be-9d3c-99ee55744b1a,30266171-963e-44a1-8861-3e281eb54e8e,2019-07-17 12:33:14,0,25036a46-d535-427f-b09a-c254e58429eb,2019-07-17,None,0,25036a46-d535-427f-b09a-c254e58429eb
4,000dcacb-bec0-4a95-8fda-777b855afb39,ad8fea0c-1a6a-4d78-a8da-a3f9807abefc,2019-06-01 01:02:47,0,d5812250-d991-44a0-a05d-98ae14b0be95,2019-06-01,None,0,d5812250-d991-44a0-a05d-98ae14b0be95


In [106]:
df[(df.buy_2 == 1) & (df.buy == 1) & (~df.rhat_merchant_id.isna())]

,session_id,account_id,click_timestamp,buy,merchant_id,dt_partition,rhat_merchant_id,buy_2,merchant_id_2
364,029f862e-9a7e-4d97-87d2-006c40501891,2d6f93b9-10f0-4e7e-a476-75ab3e33b384,2019-07-27 15:23:01,1,bdc1a67c-f0be-40a5-8594-242c419e74ce,2019-07-27,bdc1a67c-f0be-40a5-8594-242c419e74ce,1,bdc1a67c-f0be-40a5-8594-242c419e74ce
1339,09b8f149-8923-4a7d-82fa-7e863f32f8bd,56308fe2-fe1d-4bb3-89f4-bc263a18687a,2019-07-25 14:05:34,1,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,2019-07-25,bf97a8ce-be10-4ced-8de9-0ffefe3b688e,1,bf97a8ce-be10-4ced-8de9-0ffefe3b688e
2939,153c373b-e8d2-43a5-bcaf-fad45d94d067,6122925e-916a-4c93-b46b-242fcb3e6437,2019-07-30 20:32:48,1,e99d0cbe-c311-49fa-84d6-9e8406ef184a,2019-07-30,e99d0cbe-c311-49fa-84d6-9e8406ef184a,1,e99d0cbe-c311-49fa-84d6-9e8406ef184a
3465,190e754b-00be-4f0a-9549-b94d0e383a9f,c6c16677-c9b9-422a-ac26-a436e48fb7a5,2019-07-31 12:45:48,1,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992,2019-07-31,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992,1,4fb45c04-0ffa-4a2a-9f4f-9eda441b0992


In [ ]:
.withColumn('Id_New',when(df.Rank <= 5,df.Id).otherwise('other'))\


In [43]:
joined.count()

7255085

In [45]:
joined.filter(joined.rhat_merchant_id.isNotNull()).count()

68311

In [ ]:
joined.count()

In [107]:
joined.columns

['session_id',
 'account_id',
 'click_timestamp',
 'buy',
 'merchant_id',
 'dt_partition',
 'rhat_merchant_id']

In [23]:
df_info_session.filter(df_info_session.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").sort('click_timestamp').show(50)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:43|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:43|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:58|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 17:59:58|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:20|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:20|  1|  2019-07-30|
|511695b1-1a84-482...|2659f5e6-4e39-4a9...|01728dcb-857d-4a2...|2019-07-30 18:00:2

In [24]:
df_eval.filter(df_eval.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").select('account_id', 'click_timestamp', 'click_timestamp_utc').sort('click_timestamp').show(50)

+--------------------+-------------------+-------------------+
|          account_id|    click_timestamp|click_timestamp_utc|
+--------------------+-------------------+-------------------+
|2659f5e6-4e39-4a9...|2019-07-30 20:59:43|2019-07-30 17:59:43|
|2659f5e6-4e39-4a9...|2019-07-30 20:59:58|2019-07-30 17:59:58|
|2659f5e6-4e39-4a9...|2019-07-30 21:00:20|2019-07-30 18:00:20|
|2659f5e6-4e39-4a9...|2019-07-30 21:00:29|2019-07-30 18:00:29|
|2659f5e6-4e39-4a9...|2019-07-30 21:03:50|2019-07-30 18:03:50|
|2659f5e6-4e39-4a9...|2019-07-30 21:04:49|2019-07-30 18:04:49|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:35|2019-07-30 18:05:35|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:46|2019-07-30 18:05:46|
|2659f5e6-4e39-4a9...|2019-07-30 21:05:51|2019-07-30 18:05:51|
|2659f5e6-4e39-4a9...|2019-07-30 21:06:05|2019-07-30 18:06:05|
|2659f5e6-4e39-4a9...|2019-07-30 21:06:56|2019-07-30 18:06:56|
|2659f5e6-4e39-4a9...|2019-07-30 21:08:06|2019-07-30 18:08:06|
+--------------------+-------------------+-------------

In [25]:
df_order.filter(df_order.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b").select('account_id', 'click_timestamp').sort('click_timestamp').show(50)

+--------------------+-------------------+
|          account_id|    click_timestamp|
+--------------------+-------------------+
|2659f5e6-4e39-4a9...|2019-07-30 17:59:43|
|2659f5e6-4e39-4a9...|2019-07-30 17:59:58|
|2659f5e6-4e39-4a9...|2019-07-30 18:00:20|
|2659f5e6-4e39-4a9...|2019-07-30 18:00:29|
|2659f5e6-4e39-4a9...|2019-07-30 18:03:50|
|2659f5e6-4e39-4a9...|2019-07-30 18:04:49|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:35|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:46|
|2659f5e6-4e39-4a9...|2019-07-30 18:05:51|
|2659f5e6-4e39-4a9...|2019-07-30 18:06:05|
|2659f5e6-4e39-4a9...|2019-07-30 18:06:56|
|2659f5e6-4e39-4a9...|2019-07-30 18:08:06|
+--------------------+-------------------+



In [26]:
_df_eval = pd.read_csv('../output/evaluation/EvaluateAutoEncoderIfoodModel/results/VariationalAutoEncoderTraining_selu____500_fc62ac744a_6534ac1232/orders_with_metrics.csv')
_df_eval.head(2)
_df_eval[_df_eval.session_id == "511695b1-1a84-482e-af81-86f3cb05bd9b"].sort_values('click_timestamp')

,Unnamed: 0,session_id,account_id,click_timestamp,account_idx,merchant_idx,rhat_merchant_idx,shift_idx,day_of_week,rhat_scores,...,count_buys,rhat_rewards,rewards,average_precision,precision_at_1,ndcg_at_5,ndcg_at_10,ndcg_at_15,ndcg_at_20,ndcg_at_50
16439,16439,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 20:59:43,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25519,25519,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 20:59:58,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29045,29045,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:00:20,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6867,6867,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:00:29,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9243,9243,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:03:50,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12828,12828,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:04:49,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5595,5595,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:35,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:46,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15103,15103,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:05:51,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8467,8467,511695b1-1a84-482e-af81-86f3cb05bd9b,2659f5e6-4e39-4a95-ad47-a627e7dea81e,2019-07-30 21:06:05,3135,21,469,3,2,0.0,...,67845,8.918724,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_eval.columns

Index(['Unnamed: 0', 'session_id', 'account_idx', 'merchant_idx',
       'rhat_merchant_idx', 'shift_idx', 'day_of_week', 'rhat_scores', 'ps',
       'ps_eval', 'count_visits', 'count_buys', 'rhat_rewards', 'rewards',
       'average_precision', 'precision_at_1', 'ndcg_at_5', 'ndcg_at_10',
       'ndcg_at_15', 'ndcg_at_20', 'ndcg_at_50'],
      dtype='object')

In [6]:
df_eval.shape

(34940, 23)

In [11]:
df_eval.groupby(['session_id', 'account_id', 'click_timestamp']).count().shape

(34940, 20)